In [118]:
import json

tax_calculation_rules = [
    {
        "name": "Calculate Food Tax (7%)",
        "conditions": { "all": [] }, # Empty conditions mean the actions always run
        "actions": [
            {
                "name": "calculate_tax_on_variable",
                "params": {
                    "input_variable_name": "net_food_cost",
                    "tax_rate": 0.07,
                    "output_variable_name": "food_tax"
                }
            }
        ]
    },
    {
        "name": "Calculate Drink Tax (19%)",
        "conditions": { "all": [] },
        "actions": [
            {
                "name": "calculate_tax_on_variable",
                "params": {
                    "input_variable_name": "net_drink_cost",
                    "tax_rate": 0.19,
                    "output_variable_name": "drink_tax"
                }
            }
        ]
    },
    {
        "name": "Calculate Final Bill",
        "conditions": { "all": [] },
        "actions": [
            { "name": "calculate_total_bill" }
        ]
    }
]

print("---  TAX LAW RULESET LOADED ---")
for rule in tax_calculation_rules:
    print(f"\nRULE: '{rule['name']}'")
    action = rule['actions'][0]
    if action['name'] == 'calculate_total_bill':
        print("FORMULA: total_bill = net_food_cost + net_drink_cost + food_tax + drink_tax")
    else:
        params = action['params']
        print(f"FORMULA: {params['output_variable_name']} = {params['input_variable_name']} * {params['tax_rate']:.0%}")

---  TAX LAW RULESET LOADED ---

RULE: 'Calculate Food Tax (7%)'
FORMULA: food_tax = net_food_cost * 7%

RULE: 'Calculate Drink Tax (19%)'
FORMULA: drink_tax = net_drink_cost * 19%

RULE: 'Calculate Final Bill'
FORMULA: total_bill = net_food_cost + net_drink_cost + food_tax + drink_tax


In [119]:

user_orders = [
    "We had two spring rolls at 15 euros each and a large beer for 5 euros.",
    "My order was one salad for 12 euro and a bottle of water for 4 euro.",
    "The total for our food was 50 euros, and we had a bottle of wine for 25. The service was great, so add a 10 euro tip to the bill."
]

print(f"✅ List of {len(user_orders)} user orders defined successfully.")

✅ List of 3 user orders defined successfully.


In [120]:


from business_rules.variables import BaseVariables, numeric_rule_variable
from business_rules.actions import BaseActions, rule_action
from business_rules.engine import run_all
import random

class BillVariables(BaseVariables):
    def __init__(self, llm_data):
        self.data = llm_data
    @numeric_rule_variable
    def net_food_cost(self): return self.data.get('net_food_cost', 0)
    @numeric_rule_variable
    def net_drink_cost(self): return self.data.get('net_drink_cost', 0)

class BillActions(BaseActions):
    def __init__(self, variables):
        self.variables = variables
        self.calculated_taxes = {}
        self.final_bill = 0
    @rule_action(params=[
        {'fieldType': 'text', 'name': 'input_variable_name'},
        {'fieldType': 'numeric', 'name': 'tax_rate'},
        {'fieldType': 'text', 'name': 'output_variable_name'}
    ])
    def calculate_tax_on_variable(self, input_variable_name, tax_rate, output_variable_name):
        base_cost = getattr(self.variables, input_variable_name)()
        tax_amount = base_cost * tax_rate
        self.calculated_taxes[output_variable_name] = tax_amount
    @rule_action()
    def calculate_total_bill(self):
        net_food = self.variables.net_food_cost()
        net_drink = self.variables.net_drink_cost()
        total_tax = sum(self.calculated_taxes.values())
        self.final_bill = net_food + net_drink + total_tax

print("✅ Engine connector classes for Bill Calculation defined successfully.")

✅ Engine connector classes for Bill Calculation defined successfully.


In [121]:
from ipywidgets import Textarea, VBox, Button, Output, IntProgress, Layout
from IPython.display import display, clear_output, HTML
import requests
import time
import traceback
import re

# --- This is the final, encapsulated version of the demo ---

def create_tax_demo():
    """
    This function creates and displays the entire interactive demo.
    Encapsulating it helps prevent the double-rendering bug.
    """
    
    # --- UI Component Definitions ---
    query_input = Textarea(
        value=user_orders[0],
        placeholder='Paste an order here...',
        description='Customer Order:',
        style={'description_width': 'initial'},
        layout={'width': '95%', 'height': '100px'}
    )
    run_button = Button(description='Calculate Bill and Validate', button_style='success', icon='play')
    progress_bar = IntProgress(value=0, min=0, max=10, description='Waiting...', bar_style='info', orientation='horizontal', layout={'visibility': 'hidden'})
    output_area = Output()

    # --- Helper function to find JSON ---
    def extract_last_json_from_string(text):
        json_objects = re.findall(r'\{[^{}]*\}', text)
        return json_objects[-1] if json_objects else None

    # --- Main function that runs on button click ---
    def run_live_demo(button):
        with output_area:
            clear_output(wait=True)
            run_button.disabled = True
            progress_bar.description = 'Sending...'
            progress_bar.layout.visibility = 'visible'
            
            user_query = query_input.value
            prompt = f"""
            You are a restaurant billing assistant. Your tasks are:
            1. Extract the 'net_food_cost' and 'net_drink_cost'.
            2. Attempt to calculate the 'total_tax' (food tax is 7%, drink tax is 19%).
            3. Attempt to calculate the 'final_bill'.
            4. Respond ONLY with a JSON object containing these four fields.
            User Order: "{user_query}"
            JSON Response:
            """
            
            llm_raw_output = ""
            try:
                api_response_stream = requests.post('http://localhost:11434/api/generate', json={ "model": "llama3:8b", "prompt": prompt, "stream": True }, stream=True)
                api_response_stream.raise_for_status()
                progress_bar.description = 'LLM Reasoning...'; progress_bar.value = 5
                
                llm_raw_output = "".join(json.loads(chunk).get('response', '') for chunk in api_response_stream.iter_lines() if chunk)
                
                progress_bar.value = 10
                
                if not llm_raw_output.strip():
                    raise ValueError("The LLM returned an empty response.")

                llm_json_string = extract_last_json_from_string(llm_raw_output)
                if not llm_json_string:
                    raise json.JSONDecodeError("Could not find a valid JSON object in the LLM's response.", llm_raw_output, 0)

                llm_data = json.loads(llm_json_string)
                
                variables = BillVariables(llm_data)
                actions = BillActions(variables)
                actions.calculate_tax_on_variable('net_food_cost', 0.07, 'food_tax')
                actions.calculate_tax_on_variable('net_drink_cost', 0.19, 'drink_tax')
                actions.calculate_total_bill()
                
                dsl_food, dsl_drink, dsl_tax, dsl_total = variables.net_food_cost(), variables.net_drink_cost(), sum(actions.calculated_taxes.values()), actions.final_bill
                llm_food, llm_drink, llm_tax, llm_total = llm_data.get('net_food_cost', 0), llm_data.get('net_drink_cost', 0), llm_data.get('total_tax', 0), llm_data.get('final_bill', 0)
                food_match, drink_match, tax_match, total_match = ("✅" if abs(llm_food - dsl_food) < 0.01 else "❌"), ("✅" if abs(llm_drink - dsl_drink) < 0.01 else "❌"), ("✅" if abs(llm_tax - dsl_tax) < 0.01 else "❌"), ("✅" if abs(llm_total - dsl_total) < 0.01 else "❌")
                
                display(HTML("<h4>Calculation Results</h4>"))
                comparison_html = f"""
                <table style="width:100%; border-collapse: collapse;">
                  <tr style="background-color:#f2f2f2;"><th style="text-align:left; padding:8px; border: 1px solid #ddd;">Component</th><th style="text-align:right; padding:8px; border: 1px solid #ddd;">LLM Calculated</th><th style="text-align:right; padding:8px; border: 1px solid #ddd;">DSL Enforced</th><th style="text-align:center; padding:8px; border: 1px solid #ddd;">Status</th></tr>
                  <tr><td style="padding:8px; border: 1px solid #ddd;">Net Food Cost</td><td style="text-align:right; padding:8px; border: 1px solid #ddd;">€{llm_food:.2f}</td><td style="text-align:right; padding:8px; border: 1px solid #ddd;">€{dsl_food:.2f}</td><td style="text-align:center; padding:8px; border: 1px solid #ddd;">{food_match}</td></tr>
                  <tr><td style="padding:8px; border: 1px solid #ddd;">Net Drink Cost</td><td style="text-align:right; padding:8px; border: 1px solid #ddd;">€{llm_drink:.2f}</td><td style="text-align:right; padding:8px; border: 1px solid #ddd;">€{dsl_drink:.2f}</td><td style="text-align:center; padding:8px; border: 1px solid #ddd;">{drink_match}</td></tr>
                  <tr><td style="padding:8px; border: 1px solid #ddd;">Total Tax</td><td style="text-align:right; padding:8px; border: 1px solid #ddd;">€{llm_tax:.2f}</td><td style="text-align:right; padding:8px; border: 1px solid #ddd;">€{dsl_tax:.2f}</td><td style="text-align:center; padding:8px; border: 1px solid #ddd;">{tax_match}</td></tr>
                  <tr style="font-weight:bold; background-color:#f2f2f2;"><td style="padding:8px; border: 1px solid #ddd;">FINAL BILL</td><td style="text-align:right; padding:8px; border: 1px solid #ddd;">€{llm_total:.2f}</td><td style="text-align:right; padding:8px; border: 1px solid #ddd;">€{dsl_total:.2f}</td><td style="text-align:center; padding:8px; border: 1px solid #ddd;">{total_match}</td></tr>
                </table>
                """
                display(HTML(comparison_html))
                
                summary_html = "<h4>Outcome Summary</h4>"
                if total_match == "✅":
                    summary_html += "<p style='color:green;'><b>Match:</b> The LLM's calculation was compliant with all DSL rules.</p>"
                else:
                    summary_html += "<p style='color:red;'><b>Mismatch Detected:</b> The LLM's calculation was incorrect. The following errors were found:</p><ul>"
                    if food_match == "❌": summary_html += "<li>The LLM calculated the <b>Net Food Cost</b> incorrectly.</li>"
                    if drink_match == "❌": summary_html += "<li>The LLM calculated the <b>Net Drink Cost</b> incorrectly.</li>"
                    if tax_match == "❌": summary_html += "<li>The LLM calculated the <b>Total Tax</b> incorrectly.</li>"
                    summary_html += "</ul><p>The DSL-enforced calculation is the correct result.</p>"
                display(HTML(summary_html))

            except (json.JSONDecodeError, ValueError) as e:
                print("🔴 ERROR: The LLM did not return a valid or complete JSON object.")
                print("   This is a 'hallucination' where the LLM failed to follow instructions. The system caught the error.")
                print("\n--- LLM's Actual (Broken) Output ---")
                print(llm_raw_output)
            except Exception:
                print("🔴 An unexpected error occurred. Full traceback below:")
                traceback.print_exc()
            finally:
                run_button.disabled = False
                progress_bar.layout.visibility = 'hidden'
                progress_bar.value = 0

    # --- Link the function to the button and display the UI ---
    run_button.on_click(run_live_demo)
    print("--- LIVE GERMAN TAX CALCULATION DEMO ---")
    display(VBox([query_input, run_button, progress_bar, output_area]))

# --- Run the function to display the demo ---
create_tax_demo()

--- LIVE GERMAN TAX CALCULATION DEMO ---
